In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from cmdstanpy import CmdStanModel
import pandas as pd
import json
import pickle
import os

# Preprocessing data

In [2]:

df_raw = pd.read_csv('data/pilot/raw/20_01_2025/tom_dictator_v2_2025-01-20.csv')
                     
# Replace full stop with underscore in df_raw column names
df_raw.columns = df_raw.columns.str.replace('.', '_')

sessions = [
    'tgvqauld',
    'ta176utg'
]

# Only keep rows where session_id is in sessions and where participant_label is not NaN
df_full = df_raw[df_raw['session_code'].isin(sessions) & df_raw['participant_label'].notna() & (df_raw['participant__current_page_name'] == 'End')]
# Insert 
columns = ['participant_code', 'participant_label', 'session_code',
       
       'player_block_type', 'player_block_idx', 'player_idx_in_block', 'player_trust_cond', 'player_trust_label',
       'player_intention', 'player_interest', 
       'player_certainty', 'player_certainty_val', 'player_rigidity', 'player_rigidity_val',
       'player_k_lvl', 'player_trust_predicted', 'player_alpha_prior',
       'player_alpha_prior_entropy', 'player_alpha', 'player_alpha_entropy',
       'player_beta_prior', 'player_beta_prior_entropy', 'player_beta',
       'player_beta_entropy', 'player_beta_social_prior',
       'player_beta_social_prior_entropy', 'player_beta_social',
       'player_beta_social_entropy', 'player_path_taken',
       'player_perceived_certainty_certain',
       'player_perceived_certainty_uncertain',
       'player_perceived_certainty_immutable', 'player_attention',
       'player_attention_passed', 'player_attention_correct',

       'player_focus_work_high', 'player_focus_work_medium', 'player_focus_work_low',
       'player_strategy_high', 'player_strategy_low', 'player_strategy_medium',

       'subsession_round_number',
       'participant__current_app_name',
       'participant__current_page_name', 
       'player_payoff',
]


df = df_full[columns]

# If player is in column name, remove player_ from column name
df.columns = df.columns.str.replace('player_', '')


df_long = df[(df.block_type == 'test') & (df.trust_cond != 'none')][['participant_code', 'intention', 'certainty', 'rigidity', 'intention', 'trust_cond', 'path_taken']]

# Get dummies for path taken and concat with df_long
path_taken_dummies = pd.get_dummies(df_long['path_taken'], prefix='', prefix_sep='')
df_long = pd.concat([df_long, path_taken_dummies], axis=1)

df_long['delivered'] = df_long['A'] + df_long['B'] > 0
df_long['path_hidden'] = df_long['B'] + df_long['C'] > 0

# Create path_idx column
path_dict = {
    'A': 1,
    'B': 2,
    'C': 3,
    'D': 4,
}
df['path_idx'] = df['path_taken'].map(lambda x: path_dict[x])
df_long['path_idx'] = df_long['path_taken'].map(lambda x: path_dict[x])

# Create certainty_lvl column
certainties_dict = {
    'uncertain': 1,
    'certain': 2,
    'immutable': 3,
}
df_long['certainty_lvl'] = df_long['certainty'].map(lambda x: certainties_dict[x])
df['certainty_lvl'] = df['certainty'].map(lambda x: certainties_dict[x])

# Create trust_lvl column
trust_dict = {
    'none': 4,
    'low': 1,
    'medium': 2,
    'high': 3,
}
df_long['trust_lvl'] = df_long['trust_cond'].map(lambda x: trust_dict[x])
df['trust_lvl'] = df['trust_cond'].map(lambda x: trust_dict[x])

# Assign unique integer to each participant and make sure it is the same for both df and df_long and recoverable
df_long['part_idx'] = pd.Categorical(df_long['participant_code']).codes
df['part_idx'] = pd.Categorical(df['participant_code']).codes


paths_colors = {
    'A': np.array([238, 154, 86])/255,
    'B': np.array([183, 117, 63])/255,
    'C': np.array([50, 114, 169])/255,
    'D': np.array([88, 182, 225])/255,
}

/var/folders/hh/c12d0kb97ln3mqms4k45kqwm0000gn/T/ipykernel_27826/1822092935.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['path_idx'] = df['path_taken'].map(lambda x: path_dict[x])
/var/folders/hh/c12d0kb97ln3mqms4k45kqwm0000gn/T/ipykernel_27826/1822092935.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['certainty_lvl'] = df['certainty'].map(lambda x: certainties_dict[x])
/var/folders/hh/c12d0kb97ln3mqms4k45kqwm0000gn/T/ipykernel_27826/1822092935.py:82: SettingWithCopyWarning: 
A value is 

# Construct stan dataset

In [3]:
# Pivot df_wide on block_idx after keeping only path_idx as a column
stan_data = dict()

# Get only test blocks and keep only part_idx, certainty_lvl, trust_lvl, path_idx and block_idx
df_wide = df[df.block_type == 'test'][['part_idx', 'certainty_lvl', 'trust_lvl', 'path_idx', 'block_idx']].dropna()

# Pivot df_wide on block_idx for path_idx
df_path_idx = df_wide[['part_idx', 'path_idx', 'block_idx']].dropna()
df_path_idx = df_path_idx.pivot(index='part_idx', columns='block_idx', values='path_idx')
stan_data['y'] = df_path_idx.to_numpy().tolist()

# Pivot df_wide on block_idx for certainty_lvl
df_certainty = df_wide[['part_idx', 'certainty_lvl', 'block_idx']].dropna()
df_certainty = df_certainty.pivot(index='part_idx', columns='block_idx', values='certainty_lvl')
stan_data['certainties'] = df_certainty.to_numpy().tolist()

# Pivot df_wide on block_idx for trust_lvl
df_trust = df_wide[['part_idx', 'trust_lvl', 'block_idx']].dropna()
df_trust = df_trust.pivot(index='part_idx', columns='block_idx', values='trust_lvl')
stan_data['trusts'] = df_trust.to_numpy().tolist()

# Meta data
stan_data['M'], stan_data['N'] = df_path_idx.shape
stan_data['A'] = df_path_idx.nunique().max()
stan_data['C'] = df_certainty.nunique().max()
stan_data['T'] = df_trust.nunique().max()

# Stan data to json
json.dump(stan_data, open('stan_files/tom_model_basic_data.json', 'w'))

# Compile the model

In [4]:
variants = [
    #'full',
    #'onlyalpha',
    #'nobeta'
    #'nodelta' 
]
variant = variants[0]

stan_file = os.path.join('.', f'tom_model_basic_{variant}.stan')
print(stan_file)
model = CmdStanModel(stan_file=stan_file)

20:48:21 - cmdstanpy - INFO - compiling stan file /Users/vbtesh/Projects/theory_of_mind_models/stan_files/tom_model_basic_fulltemp.stan to exe file /Users/vbtesh/Projects/theory_of_mind_models/stan_files/tom_model_basic_fulltemp


./stan_files/tom_model_basic_fulltemp.stan


20:48:33 - cmdstanpy - INFO - compiled model executable: /Users/vbtesh/Projects/theory_of_mind_models/stan_files/tom_model_basic_fulltemp


# Fit the model

In [8]:
data_file = os.path.join('.', 'stan_files', 'tom_model_basic_data.json')

fit = model.sample(
    data=data_file,  
    chains=4,
    parallel_chains=4,
    iter_warmup=2500,
    iter_sampling=1000
)

21:21:54 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

21:33:26 - cmdstanpy - INFO - CmdStan done processing.
21:33:26 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: categorical_logit_lpmf: log odds parameter[1] is -inf, but must be finite! (in '/Users/vbtesh/Projects/theory_of_mind_models/stan_files/tom_model_basic_fulltemp.stan', line 173, column 12 to column 51)
Exception: categorical_logit_lpmf: log odds parameter[1] is -inf, but must be finite! (in '/Users/vbtesh/Projects/theory_of_mind_models/stan_files/tom_model_basic_fulltemp.stan', line 173, column 12 to column 51)
Exception: categorical_logit_lpmf: log odds parameter[1] is -inf, but must be finite! (in '/Users/vbtesh/Projects/theory_of_mind_models/stan_files/tom_model_basic_fulltemp.stan', line 173, column 12 to column 51)
Consider re-running with show_console=True if the above output is unclear!
21:33:26 - cmdstanpy - WARNING - Some chains may have failed to converge.
	Chain 1 had 6 divergent transitions (0.6%)
	Chain 2 had 3 divergent transitions (0.3%)
	Ch

In [9]:
print(fit.diagnose())

Processing csv files: /var/folders/hh/c12d0kb97ln3mqms4k45kqwm0000gn/T/tmpex6q0hmc/tom_model_basic_fulltempjpnmr6tg/tom_model_basic_fulltemp-20250123212154_1.csv, /var/folders/hh/c12d0kb97ln3mqms4k45kqwm0000gn/T/tmpex6q0hmc/tom_model_basic_fulltempjpnmr6tg/tom_model_basic_fulltemp-20250123212154_2.csv, /var/folders/hh/c12d0kb97ln3mqms4k45kqwm0000gn/T/tmpex6q0hmc/tom_model_basic_fulltempjpnmr6tg/tom_model_basic_fulltemp-20250123212154_3.csv, /var/folders/hh/c12d0kb97ln3mqms4k45kqwm0000gn/T/tmpex6q0hmc/tom_model_basic_fulltempjpnmr6tg/tom_model_basic_fulltemp-20250123212154_4.csv

Checking sampler transitions treedepth.
Treedepth satisfactory for all transitions.

Checking sampler transitions for divergences.
17 of 4000 (0.42%) transitions ended with a divergence.
These divergent transitions indicate that HMC is not fully able to explore the posterior distribution.
Try increasing adapt delta closer to 1.
If this doesn't remove all divergences, try to reparameterize the model.

Checking E

In [10]:
# Pickle the model
with open(f'stan_files/tom_model_basic_{variant}_model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Pickle the fit
with open(f'stan_files/tom_model_basic_{variant}_fit.pkl', 'wb') as f:
    pickle.dump(fit, f)

# Inspect the results

In [8]:
df_summary = fit.summary()
quantities = model.generate_quantities(data=data_file, previous_fit=fit)
draws = quantities.draws_pd()
df_summary.head(20)

14:38:46 - cmdstanpy - INFO - Chain [1] start processing
14:38:46 - cmdstanpy - INFO - Chain [2] start processing
14:38:46 - cmdstanpy - INFO - Chain [3] start processing
14:38:46 - cmdstanpy - INFO - Chain [4] start processing
14:38:48 - cmdstanpy - INFO - Chain [2] done processing
14:38:48 - cmdstanpy - INFO - Chain [1] done processing
14:38:48 - cmdstanpy - INFO - Chain [3] done processing
14:38:48 - cmdstanpy - INFO - Chain [4] done processing
14:38:49 - cmdstanpy - WARNING - Sample doesn't contain draws from warmup iterations, rerun sampler with "save_warmup=True".


,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
lp__,-1337.900000,0.811238,18.808900,-1369.390000,-1337.860000,-1307.220000,537.567,2.11525,1.003400
alpha_g,0.105422,0.000658,0.047856,0.038423,0.098628,0.193310,5294.500,20.83310,0.999340
sigma_alpha,0.533655,0.003159,0.088121,0.395704,0.528727,0.685627,778.025,3.06141,1.002950
beta_g,0.197632,0.000720,0.044941,0.125501,0.196526,0.273048,3897.340,15.33550,0.999631
sigma_beta,0.301687,0.001660,0.052168,0.219989,0.298625,0.388932,988.138,3.88818,1.000020
prior_inv_temps_C[1],-0.004981,0.012711,0.999616,-1.620740,0.014677,1.619690,6184.450,24.33490,1.000020
prior_inv_temps_C[2],0.006238,0.012299,0.980562,-1.588720,-0.010438,1.637950,6355.990,25.00990,0.999551
prior_inv_temps_C[3],0.010524,0.012189,1.002510,-1.628700,0.007804,1.683350,6764.540,26.61750,1.000150
alpha_M[1],-0.142701,0.009526,0.673278,-1.279810,-0.137529,0.955238,4995.030,19.65470,0.999667
alpha_M[2],0.309319,0.009627,0.672552,-0.798544,0.305184,1.444120,4880.090,19.20250,0.999797
